In [56]:
type ast = 
  | Num of int 
  | Add of ast * ast 
  | Sub of ast * ast 
  | Mul of ast * ast 
  | Div of ast * ast 
  | IfNZero of ast * ast * ast

type ast =
    Num of int
  | Add of ast * ast
  | Sub of ast * ast
  | Mul of ast * ast
  | Div of ast * ast
  | IfNZero of ast * ast * ast


In [57]:
let example_1 = IfNZero (Num 1, Num 3, Num 4)
let example_2 = Add (Num 1, Num 2)
let example_3 = Add (Num 1, IfNZero (Sub (Num 1, Num 1), Num 3, Num 4))

val example_1 : ast = IfNZero (Num 1, Num 3, Num 4)


val example_2 : ast = Add (Num 1, Num 2)


val example_3 : ast = Add (Num 1, IfNZero (Sub (Num 1, Num 1), Num 3, Num 4))


In [58]:
let _ = eval (Add(Num 1, Mul (Num 2, Num 3)))

let _ = eval (Num 1) + eval (Mul (Num 2, Num 3))

let _ = 1 + eval (Mul (Num 2, Num 3))

let _ = 1 + (eval (Num 2) * eval (Num 3))

let _ = 1 + (2 * eval (Num 3))

let _ = 1 + (2 * 3)

let _ = 1 + 6

let _ = 7



- : int = 7


- : int = 7


- : int = 7


- : int = 7


- : int = 7


- : int = 7


- : int = 7


- : int = 7


In [59]:
let rec eval = function
  | Num n -> n
  | Add (a, b) -> eval a + eval b
  | Sub (a, b) -> eval a - eval b
  | Mul (a, b) -> eval a * eval b
  | Div (a, b) -> eval a / eval b
  | IfNZero (a, b, c) -> if eval a = 0 then eval c else eval b

val eval : ast -> int = <fun>


In [60]:
let _ = eval example_1
let _ = eval example_2
let _ = eval example_3


- : int = 3


- : int = 3


- : int = 5


In [61]:
let counter = ref 0 

let new_label () = 
    let l = !counter in 
    (counter := !counter + 1; "L" ^ string_of_int l ^ "_")

type instruction =
  | SAdd
  | SSub
  | SMul
  | SDiv
  | SPush of int
  | SPop
  | SDup
  | SOver
  | SJmp of string
  | SJz of string
  | SJnz of string
  | SCmp
  | SSwp
  | SReturn
  | SLabel of string

let unparse_s = function
  | SAdd -> "add"
  | SSub -> "sub"
  | SMul -> "mul"
  | SDiv -> "div"
  | SPush n -> "push " ^ string_of_int n
  | SPop -> "pop"
  | SDup -> "dup"
  | SOver -> "over"
  | SJmp l -> "jmp " ^ l
  | SJz l -> "jz " ^ l
  | SJnz l -> "jnz " ^ l
  | SCmp -> "cmp"
  | SSwp -> "swp"
  | SReturn -> "return"
  | SLabel l -> l ^ ":"

let unparse_l l = List.map unparse_s l



val counter : int ref = {contents = 0}


val new_label : unit -> string = <fun>


type instruction =
    SAdd
  | SSub
  | SMul
  | SDiv
  | SPush of int
  | SPop
  | SDup
  | SOver
  | SJmp of string
  | SJz of string
  | SJnz of string
  | SCmp
  | SSwp
  | SReturn
  | SLabel of string


val unparse_s : instruction -> string = <fun>


val unparse_l : instruction list -> string list = <fun>


In [62]:
let rec compile = function 
  | Num n -> [SPush n]
  | Add (e1, e2) -> compile e1 @ compile e2 @ [SAdd]
  | Sub (e1, e2) -> compile e1 @ compile e2 @ [SSub]
  | Mul (e1, e2) -> compile e1 @ compile e2 @ [SMul]
  | Div (e1, e2) -> compile e1 @ compile e2 @ [SDiv]
  | IfNZero (b, e1, e2) -> 
      let l = new_label () in 
      compile b @ [SJz (l ^ "else")] @ 
      compile e1 @ [SJmp (l ^ "end"); SLabel (l ^ "else")] @ compile e2 @ [SLabel (l ^ "end")]

val compile : ast -> instruction list = <fun>


In [63]:
let _ = compile example_1
let _ = compile example_2
let _ = compile example_3

let s = List.iter print_endline (unparse_l (compile example_3))

let _ = compile (Add (Num 1, Mul (Num 2, Num 3))) = [SPush 1; SPush 2; SPush 3; SMul; SAdd]

let _ = compile (IfNZero (Num 1, Num 3, Num 4)) = [SPush 1; SJz "L0_else"; SPush 3; SJmp "L0_end"; SLabel "L0_else"; SPush 4; SLabel "L0_end"]

- : instruction list =
[SPush 1; SJz "L0_else"; SPush 3; SJmp "L0_end"; SLabel "L0_else"; SPush 4;
 SLabel "L0_end"]


- : instruction list = [SPush 1; SPush 2; SAdd]


- : instruction list =
[SPush 1; SPush 1; SPush 1; SSub; SJz "L1_else"; SPush 3; SJmp "L1_end";
 SLabel "L1_else"; SPush 4; SLabel "L1_end"; SAdd]


push 1
push 1
push 1
sub
jz L2_else
push 3
jmp L2_end
L2_else:
push 4
L2_end:
add


val s : unit = ()


- : bool = true


- : bool = false


In [64]:
let rec pos = function
  | Num n -> n > 0
  | Add (e1, e2) -> pos e1 && pos e2
  | Sub (e1, e2) -> pos e1 && neg e2
  | Mul (e1, e2) -> pos e1 && pos e2 || neg e1 && neg e2
  | Div (e1, e2) -> pos e1 && pos e2 || neg e1 && neg e2
  | IfNZero (b, e1, e2) -> pos e1 && pos e2

and neg = function
  | Num n -> n < 0
  | Add (e1, e2) -> neg e1 && neg e2
  | Sub (e1, e2) -> neg e1 && pos e2
  | Mul (e1, e2) -> neg e1 && pos e2 || pos e1 && neg e2
  | Div (e1, e2) -> neg e1 && pos e2 || pos e1 && neg e2
  | IfNZero (b, e1, e2) -> neg e1 && neg e2


let _ = assert (true  = pos (Num 1))
let _ = assert (true  = pos (Add (Num 3, Num 1)))
let _ = assert (false = pos (Add (Num 3, Num (-1))))
let _ = assert (false = pos (Sub (Num 3, Num 1)))
let _ = assert (true  = pos (Sub (Num 3, Num (-1))))
let _ = assert (true  = pos (Mul (Num 3, Num 1)))
let _ = assert (false = pos (Mul (Num 3, Num (-1))))
let _ = assert (false = pos (Mul (Add (Num 3, Num (-1)), Num (-1))))

val pos : ast -> bool = <fun>
val neg : ast -> bool = <fun>


- : unit = ()


- : unit = ()


- : unit = ()


- : unit = ()


- : unit = ()


- : unit = ()


- : unit = ()


- : unit = ()


In [65]:
type lambda = 
  | Num of int 
  | Add of lambda * lambda 

  | Var of string 
  | App of lambda * lambda 
  | Fun of string * lambda 

let f = Fun ("x", Add (Var "x", Num 1))
let e = App (f, Num 2)

type value = 
  | IntValue of int 
  | Closure of string * lambda

let rec subst x e e' = 
  let rec subst' e' = 
    match e' with
    | Num n -> Num n
    | Add (e1, e2) -> Add (subst' e1, subst' e2)
    | Var y -> if x = y then e else e'
    | App (e1, e2) -> App (subst' e1, subst' e2)
    | Fun (y, e) -> if x = y then Fun (y, e) else Fun (y, subst' e)
  in subst' e'

let rec eval_lambda = function
  | Num n -> IntValue n
  | Add (e1, e2) -> 
      (match eval_lambda e1, eval_lambda e2 with
       | IntValue n1, IntValue n2 -> IntValue (n1 + n2)
       | _ -> failwith "Not an integer value")
  | Fun (x, e) -> Closure (x, e)
  | App (e1, e2) -> 
      (match eval_lambda e1 with
       | Closure (x, e) -> eval_lambda (subst x e2 e)
       | _ -> failwith "Not a function")
  | Var x -> failwith "Undefined variable"

type lambda =
    Num of int
  | Add of lambda * lambda
  | Var of string
  | App of lambda * lambda
  | Fun of string * lambda


val f : lambda = Fun ("x", Add (Var "x", Num 1))


val e : lambda = App (Fun ("x", Add (Var "x", Num 1)), Num 2)


type value = IntValue of int | Closure of string * lambda


val subst : string -> lambda -> lambda -> lambda = <fun>


val eval_lambda : lambda -> value = <fun>


In [66]:
(* Not to be executed, only for slide making *)

eval_lambda App (Fun ("x", Add (Var "x", Num 1)), Num 2)
  eval_lambda Fun ("x", Add (Var "x", Num 1))
    = Closure ("x", Add (Var "x", Num 1))
  eval_lambda Num 2
    = IntValue 2

  eval_lambda subst "x" (Num 2) (Add (Var "x", Num 1))
    = eval_lambda (Add (Num 2, Num 1))
    = IntValue 3

= IntValue 3


error: compile_error